# Execution Tutorial - Part 1

This tutorial covers the basics of executing a quantum program using Classiq directly through the Python SDK. It is also possible to use the [Classiq Platform](https://platform.classiq.io) to execute quantum algorithms.

For this, we will start by synthesizing the following example from the [synthesis tutorial](https://docs.classiq.io/latest/explore/tutorials/basic_tutorials/the_classiq_tutorial/synthesis_tutorial/):

In [1]:
from classiq import *


@qfunc
def main(x: Output[QNum[3]], y: Output[QNum]) -> None:
    allocate(x)
    hadamard_transform(x)
    y |= x**2 + 1


qprog = synthesize(main)
show(qprog)

Quantum program link: https://platform.classiq.io/circuit/39GIErHvatLUOma8kkt0RGTVsxU


This quantum program evaluates the function $y(x) = x^2 + 1$, for all integers $x \in [0,7]$. To execute a quantum program and save its results in the Python SDK, create an `ExecutionSession`. To sample the states using this object, one can use `sample`:

In [2]:
with ExecutionSession(qprog) as es:
    results = es.sample()

The information from the outputs of the quantum program can be obtained in the form of a dataframe using the `dataframe` attribute:

In [3]:
results.dataframe

,x,y,counts,count,probability,bitstring
0,7,50,284,284,0.138672,110010111
1,3,10,272,272,0.132812,001010011
2,4,17,271,271,0.132324,010001100
3,0,1,261,261,0.127441,000001000
4,5,26,251,251,0.122559,011010101
5,1,2,240,240,0.117188,000010001
6,6,37,235,235,0.114746,100101110
7,2,5,234,234,0.114258,000101010


The information displayed in the dataframe is:

* `counts` shows the number of times each state was measured.
* `bitstring` is the bitstring that represents each state measured.
* `x` and `y` are the numerical representation of the states associated with the measurement.
* `probability` is the probability associated with each measured state.

By default, the number of executions of the quantum program is $2048$. This quantity, called the number of shots, can be modified using `ExecutionPreferences`. For instance, if we want to execute the same circuit with $10{,}000$ shots:

In [4]:
prefs_more_shots = ExecutionPreferences(num_shots=10000)

with ExecutionSession(qprog, execution_preferences=prefs_more_shots) as es:
    results_more_shots = es.sample()

The number of counts for each state will grow proportionally with the number of shots:

In [5]:
results_more_shots.dataframe

,x,y,counts,count,probability,bitstring
0,1,2,1280,1280,0.1280,000010001
1,6,37,1272,1272,0.1272,100101110
2,2,5,1264,1264,0.1264,000101010
3,3,10,1255,1255,0.1255,001010011
4,4,17,1252,1252,0.1252,010001100
5,7,50,1251,1251,0.1251,110010111
6,0,1,1243,1243,0.1243,000001000
7,5,26,1183,1183,0.1183,011010101


## Backend selection

The backend of an execution is the hardware or simulator where the quantum program is executed. To select a specific backend, it is necessary to use its correct Backend Preferences. Check the different [Cloud Providers](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/) and their backend preferences for execution.

In this section, we will explore two different examples for clarification: 

### First example: Execution using the state vector simulator from Classiq

A state vector simulator outputs the amplitudes of a quantum program. On real hardware, obtaining these amplitudes requires quantum tomography — the process of measuring in different bases to reconstruct the output state.

Since Classiq provides its own state vector simulator backend, we will use `ClassiqBackendPreferences` to define it as the state vector simulator. This information is provided on the [Cloud Providers page](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/).

To define the quantum program's execution preferences, use `execution_preferences` under `ExecutionSession`. In this example, we will perform a simulation with `num_shots=1` since the state vector simulator performs an exact simulation of the quantum program.

If no backend is defined in the preferences, then the [Classiq simulator](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/classiq-backends/) is selected by default.


In [6]:
backend_preferences = ClassiqBackendPreferences(
    backend_name="simulator_statevector"
)  # Always check the Cloud Providers to correctly define the backend.

execution_preferences = ExecutionPreferences(
    num_shots=1, backend_preferences=backend_preferences
)

Now, execute the quantum program using `execute`.

In [7]:
with ExecutionSession(qprog, execution_preferences=execution_preferences) as es:
    results_statevector = es.sample()

The outputs of the quantum program can be displayed using the `dataframe` property:

In [8]:
df = results_statevector.dataframe
df[df["amplitude"].abs() > 0.01]

,x,y,amplitude,magnitude,phase,probability,bitstring
0,2,5,0.000000+0.353553j,0.35,0.50π,0.125,000101010
1,5,26,0.000000+0.353553j,0.35,0.50π,0.125,011010101
2,3,10,0.000000+0.353553j,0.35,0.50π,0.125,001010011
3,1,2,0.000000+0.353553j,0.35,0.50π,0.125,000010001
4,4,17,0.000000+0.353553j,0.35,0.50π,0.125,010001100
5,6,37,0.000000+0.353553j,0.35,0.50π,0.125,100101110
6,0,1,0.000000+0.353553j,0.35,0.50π,0.125,000001000
7,7,50,0.000000+0.353553j,0.35,0.50π,0.125,110010111


The outputs from the execution obtained via statevector simulator will differ from the default simulator:

* `state_vector` will output a `dict` containing the bitstrings followed by its numerically evaluated amplitudes.
* `parsed_state_vector` will output a `list` of `SimulatedState`, each containing the values of `x` and `y` followed by its bitstrings and its numerically evaluated amplitudes.